#Comparison of Test Sample vs. Prebuilt Templates (30 Phases)

This block loads a single test sample (with 30 aortic velocity frames) and compares each frame to the corresponding pre-built template created from other subjects.

The following comparisons are performed:
- **Area** of the binary mask for each test frame and its corresponding template.
- **Center of Mass (COM)** for spatial location comparison.
- **Visual Comparison**: Each test frame and its matching template are displayed side-by-side.
- **Tabular Summary**: A DataFrame with ΔArea and ΔCOM (Y/X) for each frame is generated and saved as CSV.

 *Note: The templates were constructed from a different cohort (not from the test sample), so this comparison helps evaluate how similar/dissimilar the test frames are from the generalized templates.*

Purpose: Identify if significant mismatch exists that might explain poor alignment or shrinkage during registration.


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import center_of_mass
import pandas as pd
import IPython.display as disp

# ---------------------- Settings ----------------------
# Input test velocity file (masked velocity array)
test_path = r"P:\Projects\DeepFlow\deepFlowDocker\scripts\Registration\data\20213_2_0_masked.npy"

# Folder containing 30 phase-specific templates
template_dir = r"P:\Projects\DeepFlow\deepFlowDocker\scripts\Registration\templates\Template10allsimNonrigid"
template_prefix = "template_phase"

# ---------------------- Load Test Data ----------------------
# Binary mask extraction from velocity
vel = np.load(test_path)
test_masks = (np.abs(vel) > 1e-6).astype(np.uint8)

# Load all 30 template masks
templates = [np.load(os.path.join(template_dir, f"{template_prefix}_{i:02d}_similarity_10.npy")) for i in range(30)]

# ---------------------- Compute Area + COM ----------------------
def compute_stats(stack):
    return [
        {
            "area": int(np.sum(f)),
            "com": tuple(np.round(center_of_mass(f), 2))
        }
        for f in stack
    ]

test_stats = compute_stats(test_masks)
template_stats = compute_stats(templates)

# ---------------------- Visual Comparison ----------------------
for i in range(30):
    fig, axes = plt.subplots(1, 2, figsize=(9, 5))
    
    axes[0].imshow(test_masks[i], cmap='gray')
    axes[0].set_title(f"Test Frame {i}\nArea={test_stats[i]['area']}\nCOM={test_stats[i]['com']}", fontsize=10)
    axes[0].axis('off')

    axes[1].imshow(templates[i], cmap='gray')
    axes[1].set_title(f"Template {i}\nArea={template_stats[i]['area']}\nCOM={template_stats[i]['com']}", fontsize=10)
    axes[1].axis('off')

    plt.suptitle(f"Comparison - Frame {i}", fontsize=12)
    plt.tight_layout(rect=[0, 0, 1, 0.95])  # Leaves space for suptitle
    plt.show()

# ---------------------- Stats Table ----------------------
df = pd.DataFrame([
    {
        "frame": i,
        "test_area": test_stats[i]["area"],
        "template_area": template_stats[i]["area"],
        "ΔA": test_stats[i]["area"] - template_stats[i]["area"],
        "test_COM": test_stats[i]["com"],
        "template_COM": template_stats[i]["com"]
    }
    for i in range(30)
])
df["ΔCOM_y"] = df["test_COM"].apply(lambda x: x[0]) - df["template_COM"].apply(lambda x: x[0])
df["ΔCOM_x"] = df["test_COM"].apply(lambda x: x[1]) - df["template_COM"].apply(lambda x: x[1])

# Show stats as a table
disp.display(df)

# Save to file
df.to_csv("test_vs_template_stats.csv", index=False)


In [ ]:
# ====================== Compare Test Sample vs. Prebuilt Templates =======================
# This notebook compares each frame of a test sample (e.g., 30-frame .npy velocity-masked file)
# to the corresponding prebuilt templates and optionally the aligned outputs.

import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import center_of_mass
import seaborn as sns

# ----------------------- SETTINGS -----------------------

# Path to test sample .npy file (original velocity-masked input)
test_path = r"P:\Projects\DeepFlow\deepFlowDocker\scripts\Registration\data\20213_2_0_masked.npy"

# Path to aligned output (optional - set to None to skip)
aligned_path = r"P:\Projects\DeepFlow\deepFlowDocker\scripts\Registration\output\10simNonRigidTemplateallsim\20213_2_0_masked_aligned_velocity.npy"
                 
# Template directory and file prefix
template_dir = r"P:\Projects\DeepFlow\deepFlowDocker\scripts\Registration\templates\Template10allsimNonrigid"
template_prefix = "template_phase"
template_suffix = "_similarity_10.npy"

# ----------------------- Load Data -----------------------

def load_binary_masks(file_path, threshold=1e-6):
    data = np.load(file_path)
    return (np.abs(data) > threshold).astype(np.uint8)

test_masks = load_binary_masks(test_path)
aligned_masks = load_binary_masks(aligned_path) if aligned_path else None

templates = []
for i in range(30):
    fname = f"{template_prefix}_{i:02d}{template_suffix}"
    path = os.path.join(template_dir, fname)
    tpl = np.load(path)
    tpl = (tpl > 0.5).astype(np.uint8) if tpl.max() <= 1 else tpl.astype(np.uint8)
    templates.append(tpl)
templates = np.array(templates)

# ----------------------- Analysis -----------------------

def compute_stats(mask):
    area = np.sum(mask)
    com = center_of_mass(mask) if area > 0 else (np.nan, np.nan)
    return area, com

rows = []
for i in range(30):
    row = {'Frame': i}

    area_t, com_t = compute_stats(templates[i])
    area_i, com_i = compute_stats(test_masks[i])
    
    row.update({
        'Area_template': area_t,
        'COM_template_y': com_t[0], 'COM_template_x': com_t[1],
        'Area_input': area_i,
        'COM_input_y': com_i[0], 'COM_input_x': com_i[1],
        'ΔA': area_i - area_t,
        'ΔCOM_y': com_i[0] - com_t[0],
        'ΔCOM_x': com_i[1] - com_t[1],
    })

    if aligned_masks is not None:
        area_a, com_a = compute_stats(aligned_masks[i])
        row.update({
            'Area_aligned': area_a,
            'COM_aligned_y': com_a[0], 'COM_aligned_x': com_a[1],
            'ΔA_after': area_a - area_t,
            'ΔCOM_y_after': com_a[0] - com_t[0],
            'ΔCOM_x_after': com_a[1] - com_t[1],
        })

    rows.append(row)

import pandas as pd
df = pd.DataFrame(rows)
display(df.round(2))

# ----------------------- Visualization -----------------------

def show_comparison_row(i):
    fig, axs = plt.subplots(1, 3 if aligned_masks is not None else 2, figsize=(15, 5))
    
    axs[0].imshow(test_masks[i], cmap='gray')
    axs[0].set_title(f"Input Mask - Frame {i}")
    axs[0].axis('off')

    axs[1].imshow(templates[i], cmap='gray')
    axs[1].set_title(f"Template Mask - Frame {i}")
    axs[1].axis('off')

    if aligned_masks is not None:
        axs[2].imshow(aligned_masks[i], cmap='gray')
        axs[2].set_title(f"Aligned Mask - Frame {i}")
        axs[2].axis('off')

    plt.tight_layout()
    plt.show()

# Example: visualize frames 10–20
for i in range(10, 21):
    show_comparison_row(i)


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import center_of_mass
import pandas as pd

# ----------------------- Settings -----------------------
test_path = r"P:\\Projects\\DeepFlow\\deepFlowDocker\\scripts\\Registration\\data\\20213_2_0_masked.npy"
aligned_path = r"P:\\Projects\\DeepFlow\\deepFlowDocker\\scripts\\Registration\\output\\10simNonRigidTemplateallsim\\20213_2_0_masked_aligned_velocity.npy"
template_dir = r"P:\\Projects\\DeepFlow\\deepFlowDocker\\scripts\\Registration\\templates\\Template10allsimNonrigid"
template_prefix = "template_phase"
template_suffix = "_similarity_10.npy"

output_dir = r"P:\\Projects\\DeepFlow\\deepFlowDocker\\scripts\\Registration\\notebooks\\analysis_output"
os.makedirs(output_dir, exist_ok=True)

# ----------------------- Load -----------------------
def load_binary_masks(file_path, threshold=1e-6):
    data = np.load(file_path)
    return (np.abs(data) > threshold).astype(np.uint8)

test_masks = load_binary_masks(test_path)
aligned_masks = load_binary_masks(aligned_path) if aligned_path else None

templates = []
for i in range(30):
    fname = f"{template_prefix}_{i:02d}{template_suffix}"
    path = os.path.join(template_dir, fname)
    tpl = np.load(path)
    tpl = (tpl > 0.5).astype(np.uint8) if tpl.max() <= 1 else tpl.astype(np.uint8)
    templates.append(tpl)
templates = np.array(templates)

# ----------------------- Compute Table -----------------------
def compute_stats(mask):
    area = int(np.sum(mask))
    com = center_of_mass(mask) if area > 0 else (np.nan, np.nan)
    return area, com

rows = []
for i in range(30):
    row = {'Frame': i}

    area_t, com_t = compute_stats(templates[i])
    area_i, com_i = compute_stats(test_masks[i])

    row.update({
        'Area_template': area_t,
        'COM_template_y': com_t[0], 'COM_template_x': com_t[1],
        'Area_input': area_i,
        'COM_input_y': com_i[0], 'COM_input_x': com_i[1],
        'Delta_A': int(area_i - area_t),
        'Delta_COM_y': float(com_i[0] - com_t[0]),
        'Delta_COM_x': float(com_i[1] - com_t[1]),
    })

    if aligned_masks is not None:
        area_a, com_a = compute_stats(aligned_masks[i])
        row.update({
            'Area_aligned': area_a,
            'COM_aligned_y': com_a[0], 'COM_aligned_x': com_a[1],
            'Delta_A_after': int(area_a - area_t),
            'Delta_COM_y_after': float(com_a[0] - com_t[0]),
            'Delta_COM_x_after': float(com_a[1] - com_t[1]),
            'Delta_A_input_aligned': int(area_a - area_i),
            'Delta_COM_y_input_aligned': float(com_a[0] - com_i[0]),
            'Delta_COM_x_input_aligned': float(com_a[1] - com_i[1]),
        })

    rows.append(row)

df = pd.DataFrame(rows)
df.round(2).to_csv(os.path.join(output_dir, "comparison_stats.csv"), index=False)

# ----------------------- Visualization -----------------------
n_cols = 3 if aligned_masks is not None else 2
fig, axes = plt.subplots(30, n_cols, figsize=(n_cols * 4, 30 * 1.6))

for i in range(30):
    axes[i, 0].imshow(test_masks[i], cmap='gray')
    axes[i, 0].set_title(f"Input [{i}]", fontsize=8)
    axes[i, 0].axis('off')

    axes[i, 1].imshow(templates[i], cmap='gray')
    axes[i, 1].set_title(f"Template [{i}]", fontsize=8)
    axes[i, 1].axis('off')

    if aligned_masks is not None:
        axes[i, 2].imshow(aligned_masks[i], cmap='gray')
        axes[i, 2].set_title(f"Aligned [{i}]", fontsize=8)
        axes[i, 2].axis('off')

plt.tight_layout()
figpath = os.path.join(output_dir, "all_frames_comparison.png")
plt.savefig(figpath, dpi=300)
plt.close()

print("Saved CSV:", os.path.join(output_dir, "comparison_stats.csv"))
print("Saved image:", figpath)

# ----------------------- Show Table and Image Inline -----------------------
from IPython.display import display, Image

# 1. نمایش جدول (گرد شده)
display(df.round(2))

# 2. نمایش تصویر تمام فریم‌ها در نوت‌بوک
from PIL import Image as PILImage

img_path = os.path.join(output_dir, "all_frames_comparison.png")
img = PILImage.open(img_path)
plt.figure(figsize=(12, 80))
plt.imshow(img)
plt.axis('off')
plt.title("Comparison: Input vs. Template vs. Aligned (30 Frames)", fontsize=14)
plt.show()